In [1]:
using Distributions
using LinearAlgebra
using Random
using CSV

┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.1/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.1/CSV/HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1184


In [2]:
using DataFrames, SparseArrays, LinearAlgebra
# This function returns a dictionary with the unique values in the vector "a" as the keys and their 
# sequential numbers as the associated values
# It also returns vector with the keys in sequential order. 
function mkDict(a)
  aUnique = unique(a)
  d = Dict()
  names = Array{String}(undef,size(aUnique,1))
  for (i,s) in enumerate(aUnique)
    names[i] = s
    d[s] = i
  end
  return d,names
end




function getX(factor,df;cov=false)
    data = df[:,Symbol(factor)]
    n = size(data,1)
    if cov==false
        str = data
        val = 1.0
    else 
        str = fill(factor,n)
        val = data
    end

    dict,names   = mkDict(str)
    ii    = 1:n                    # row numbers 
    jj   = [dict[i] for i in str]  # column numbers
    X    = sparse(ii,jj,val)    
end  

getX (generic function with 1 method)

In [3]:
function readMatBin(fileName)
    genStr = open(fileName)
    n = read(genStr,Int64)
    p = read(genStr,Int64)
    M = zeros(n,p)
    for j in 1:p
        for i in 1:n
            M[i,j] = read(genStr,Float64)
        end
    end
    close(genStr)
    return M
end
@time M = readMatBin("/home/jovyan/mark/JWAS_CDPQ/CDPQ_all_data/ALGP2_Cycle_1_7_Geno_Complete_Impute_JWAS.bin.gen.bin")

 28.471359 seconds (37.32 k allocations: 10.393 GiB, 0.07% gc time)


3205×435172 Array{Float64,2}:
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  2.0  2.0  0.0  1.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 1.0  1.0  0.0  1.0  2.0  0.0  0.0  2.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [4]:
using DelimitedFiles
rowID = vec(readdlm("/home/jovyan/mark/JWAS_CDPQ/CDPQ_all_data/ALGP2_Cycle_1_7_Geno_Complete_Impute_JWAS.bin.ids.txt",String))

3205-element Array{String,1}:
 "0001"
 "0002"
 "0004"
 "0005"
 "0006"
 "0007"
 "0009"
 "0010"
 "0011"
 "0012"
 "0013"
 "0014"
 "0015"
 ⋮     
 "3283"
 "3284"
 "3285"
 "3286"
 "3287"
 "3288"
 "3289"
 "3290"
 "3291"
 "3292"
 "3293"
 "3294"

In [5]:
phenotypes = CSV.read("production_1_7_Exp.dat", types=Dict(:ID => String, :Died => String), delim = ' ',header=true, missingstrings=["NA"] )
first(phenotypes,10)

,ID,Cycle,BatchNum,Batch,Company,SowID,NurEnrich,QNurPenBatch,NurPenBatch
,String,Int64,Int64,String,String,String,String,String,String⍰
1,0001,1,1,1B,A,1B_1715,FALSE,1B_D04,1B_D04
2,0002,1,1,1B,A,1B_1642,FALSE,1B_D04,1B_D04
3,0004,1,1,1B,A,1B_2150,FALSE,1B_D04,1B_D04
4,0005,1,1,1B,A,1B_1980,FALSE,1B_D04,1B_D04
5,0006,1,1,1B,A,1B_1902,FALSE,1B_D04,1B_D04
6,0007,1,1,1B,A,1B_2150,FALSE,1B_D04,1B_D04
7,0009,1,1,1B,A,1B_1902,FALSE,1B_D02,1B_D02
8,0010,1,1,1B,A,1B_1885,FALSE,1B_D02,1B_D02
9,0011,1,1,1B,A,1B_2097,FALSE,1B_D02,1B_D02


In [6]:
res = phenotypes[:,:Nur2ADG]
phenotypes=dropmissing(phenotypes, :Nur2ADG)
first(phenotypes,10)

,ID,Cycle,BatchNum,Batch,Company,SowID,NurEnrich,QNurPenBatch,NurPenBatch
,String,Int64,Int64,String,String,String,String,String,String⍰
1,0001,1,1,1B,A,1B_1715,FALSE,1B_D04,1B_D04
2,0002,1,1,1B,A,1B_1642,FALSE,1B_D04,1B_D04
3,0004,1,1,1B,A,1B_2150,FALSE,1B_D04,1B_D04
4,0005,1,1,1B,A,1B_1980,FALSE,1B_D04,1B_D04
5,0007,1,1,1B,A,1B_2150,FALSE,1B_D04,1B_D04
6,0009,1,1,1B,A,1B_1902,FALSE,1B_D02,1B_D02
7,0010,1,1,1B,A,1B_1885,FALSE,1B_D02,1B_D02
8,0011,1,1,1B,A,1B_2097,FALSE,1B_D02,1B_D02
9,0012,1,1,1B,A,1B_1902,FALSE,1B_D02,1B_D02


In [7]:
sel = .!(ismissing.(res))
ZFull = Matrix{Float64}(I,3205,3205)
Z = ZFull[sel,:]

3172×3205 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [8]:
# Add markers
varG = 0.295395E-02
sum_2pq=165899.511107207
varAlpha = varG/sum_2pq
@time G = Z*M*M'*Z'*varAlpha

 61.738259 seconds (1.35 M allocations: 10.574 GiB, 0.89% gc time)


3172×3172 Array{Float64,2}:
 0.0129128   0.0112167   0.010408    …  0.0101365  0.0100738   0.00989476
 0.0112167   0.0130351   0.0102541      0.0101309  0.0100851   0.00994304
 0.010408    0.0102541   0.012849       0.0101402  0.0100326   0.00974491
 0.0102313   0.0104692   0.0100764      0.0100464  0.0101097   0.00982027
 0.010297    0.0103345   0.0115547      0.010044   0.0100768   0.0097877 
 0.0108566   0.0111189   0.0103271   …  0.0101269  0.0100352   0.00984461
 0.010282    0.0104129   0.0107392      0.0100809  0.0100971   0.00974967
 0.0102237   0.0102356   0.0103028      0.0100304  0.00993201  0.00985399
 0.0104252   0.0104298   0.0103173      0.0101418  0.0101148   0.00987326
 0.0104348   0.010607    0.011048       0.0101178  0.0100102   0.00978132
 0.0103884   0.0103724   0.0117066   …  0.0102054  0.0100674   0.00985725
 0.0103007   0.0105695   0.0101989      0.0101569  0.0101805   0.00989209
 0.0103936   0.0105773   0.01015        0.010037   0.0101401   0.00985121
 ⋮        

In [9]:
# treat fixed effect as random with large variance, because residual var/ batch var is too small, nothing was added to diagonal.
X1 = getX("Batch",phenotypes)
X2 = getX("Died",phenotypes)
X3 = getX("EntryAge",phenotypes,cov=true)

var_Batch = 1
var_Died  = 1
var_Age=0.103
X=var_Batch*X1*X1' + var_Died*X2*X2' + var_Age*X3*X3'
Matrix(X)

3172×3172 Array{Float64,2}:
 51.852  49.586  49.586  49.586  49.586  …  65.714  66.714  66.714  66.714
 49.586  47.423  47.423  47.423  47.423     62.727  63.727  63.727  63.727
 49.586  47.423  47.423  47.423  47.423     62.727  63.727  63.727  63.727
 49.586  47.423  47.423  47.423  47.423     62.727  63.727  63.727  63.727
 49.586  47.423  47.423  47.423  47.423     62.727  63.727  63.727  63.727
 47.32   45.26   45.26   45.26   45.26   …  59.74   60.74   60.74   60.74 
 49.586  47.423  47.423  47.423  47.423     62.727  63.727  63.727  63.727
 51.852  49.586  49.586  49.586  49.586     65.714  66.714  66.714  66.714
 47.32   45.26   45.26   45.26   45.26      59.74   60.74   60.74   60.74 
 51.852  49.586  49.586  49.586  49.586     65.714  66.714  66.714  66.714
 49.586  47.423  47.423  47.423  47.423  …  62.727  63.727  63.727  63.727
 49.586  47.423  47.423  47.423  47.423     62.727  63.727  63.727  63.727
 49.586  47.423  47.423  47.423  47.423     62.727  63.727  63.727  63.7

In [10]:
Z1 = getX("NurPenBatch",phenotypes)
Matrix(Z1*Z1')
Z2 = getX("SowID",phenotypes)
Matrix(Z2*Z2')

var_pen= 0.107150E-02
var_sow= 0.927471E-03
U=var_pen*Z1*Z1' + var_sow*Z2*Z2'

3172×3172 SparseMatrixCSC{Float64,Int64} with 30240 stored entries:
  [1   ,    1]  =  0.00199897
  [2   ,    1]  =  0.0010715
  [3   ,    1]  =  0.0010715
  [4   ,    1]  =  0.0010715
  [5   ,    1]  =  0.0010715
  [33  ,    1]  =  0.000927471
  [48  ,    1]  =  0.000927471
  [1   ,    2]  =  0.0010715
  [2   ,    2]  =  0.00199897
  [3   ,    2]  =  0.0010715
  [4   ,    2]  =  0.0010715
  [5   ,    2]  =  0.0010715
  ⋮
  [3156, 3171]  =  0.0010715
  [3163, 3171]  =  0.0010715
  [3171, 3171]  =  0.00199897
  [3100, 3172]  =  0.0010715
  [3101, 3172]  =  0.0010715
  [3102, 3172]  =  0.0010715
  [3110, 3172]  =  0.0010715
  [3118, 3172]  =  0.0010715
  [3144, 3172]  =  0.0010715
  [3153, 3172]  =  0.0010715
  [3170, 3172]  =  0.0010715
  [3172, 3172]  =  0.00199897

In [11]:
varRes = 0.115504E-01
V = G + U + X + I*varRes

3172×3172 Array{Float64,2}:
 51.8785  49.5983  49.5975  49.5973  …  65.724   66.7241  66.7241  66.7239
 49.5983  47.4496  47.4343  47.4345     62.737   63.7371  63.7371  63.7369
 49.5975  47.4343  47.4494  47.4341     62.7371  63.7371  63.737   63.7367
 49.5973  47.4345  47.4341  47.4495     62.737   63.737   63.7371  63.7368
 49.5974  47.4344  47.4366  47.4342     62.737   63.737   63.7371  63.7368
 47.3309  45.2711  45.2703  45.2704  …  59.75    60.7501  60.75    60.7498
 49.5963  47.4334  47.4337  47.4333     62.737   63.7371  63.7371  63.7367
 51.8622  49.5962  49.5963  49.5963     65.724   66.724   66.7239  66.7239
 47.3304  45.2704  45.2703  45.2702     59.75    60.7501  60.7501  60.7499
 51.8624  49.5966  49.597   49.5961     65.7241  66.7241  66.724   66.7238
 49.5964  47.4334  47.4356  47.4332  …  62.7371  63.7372  63.7371  63.7369
 49.5963  47.4336  47.4332  47.4356     62.7371  63.7372  63.7372  63.7369
 49.5964  47.4336  47.4331  47.4357     62.737   63.737   63.7371  63.73

In [12]:
Vi = inv(V)
C = G
n = size(V,1)
y  = phenotypes[!,:Nur2ADG]

3172-element Array{Float64,1}:
 0.411764705882353 
 0.411764705882353 
 0.558823529411765 
 0.735294117647059 
 0.529411764705882 
 0.470588235294118 
 0.617647058823529 
 0.5               
 0.5               
 0.470588235294118 
 0.588235294117647 
 0.588235294117647 
 0.558823529411765 
 ⋮                 
 0.470344827586207 
 0.340689655172414 
 0.551034482758621 
 0.501379310344828 
 0.268965517241379 
 0.0144827586206897
 0.361379310344828 
 0.126896551724138 
 0.0937931034482759
 0.368965517241379 
 0.320689655172414 
 0.33448275862069  

In [13]:
# Calculate breeding values using BLUP
EBV_Orig=G*inv(V)*y

3172-element Array{Float64,1}:
 -0.03269978962740087  
 -0.04141103528651882  
 -0.003404559298088642 
  0.007333132302117441 
 -0.0028078252742749056
 -0.029104820328999578 
  0.0016058431852922625
 -0.029221657972404586 
 -0.028075201173814717 
 -0.029629968296047794 
 -0.0166077387581616   
 -0.009703184159034263 
  0.0092035811108483   
  ⋮                    
 -0.020597732741668706 
  0.020904890078992253 
  0.02532503793606577  
  0.01344567895642994  
 -0.01250545368331349  
 -0.038586502764992196 
 -0.014704111509553152 
 -0.09629083207824132  
 -0.031063957746966343 
 -0.005268594302331009 
 -0.0425139079205449   
 -0.004530648598336726 

In [85]:
# Inefficient way for LOOCV

function loiEBV(i)
    sel = setdiff(1:n,i)
    C[i,sel]'*inv(V[sel,sel])*y[sel]
end

loiEBV (generic function with 1 method)

In [ ]:
@time EBV=[loiEBV(i) for i=1:n]

In [13]:
# Efficient way for LOOCV
function loiEBVEff2(i,Vi,C,y)
    r     = 1/Vi[i,i]
    q     = -Vi[i,:] *r
    yi    = copy(y)
    yi[i] = 0.0
    Ci    = C[i,:]
    Ci[i] = 0.0
    (Ci'*Vi - (Ci'q) * q'/r)*yi
end

loiEBVEff2 (generic function with 1 method)

In [14]:
@time EBV2=[loiEBVEff2(i,Vi,C,y) for i=1:n]

  6.442710 seconds (1.83 M allocations: 784.153 MiB, 1.06% gc time)


3172-element Array{Float64,1}:
 -0.02134861302021401  
 -0.03207447234044558  
 -0.007818274085819409 
 -0.021120092137236878 
 -0.0030139058085089238
 -0.025453230089774376 
 -0.00954434535320289  
 -0.03094235383853869  
 -0.029033068634815862 
 -0.028465122340812624 
 -0.02705440265080996  
 -0.020456884014111807 
  0.007274276042889801 
  ⋮                    
 -0.024192794043912738 
  0.028774730282749326 
  0.014197732362687607 
  0.0030743637515259223
 -0.013397549494204332 
 -0.014317324929955222 
 -0.011063145484027637 
 -0.07106271681346113  
 -0.01535671565076456  
 -0.0002466794440884289
 -0.0347466454731071   
  0.005379190734722927 

In [18]:
function loiEBVEff(i)
    sel = setdiff(1:n,i)
    r   = 1/Vi[i,i]
    q   = -Vi[i,sel] *r
    C[i,sel]'*(Vi[sel,sel] - q*q'/r)*y[sel]
end

loiEBVEff (generic function with 1 method)

In [19]:
@time EBV1=[loiEBVEff(i) for i=1:n]

794.748713 seconds (1.21 M allocations: 951.691 GiB, 38.28% gc time)


3172-element Array{Float64,1}:
 -0.021348613020205576  
 -0.0320744723404313    
 -0.007818274085823931  
 -0.021120092137236403  
 -0.0030139058085021384 
 -0.02545323008977707   
 -0.009544345353187045  
 -0.030942353838545204  
 -0.02903306863482038   
 -0.028465122340809352  
 -0.027054402650810574  
 -0.0204568840141332    
  0.007274276042869225  
  ⋮                     
 -0.02419279404391802   
  0.028774730282750453  
  0.014197732362694147  
  0.003074363751511487  
 -0.013397549494198913  
 -0.014317324929947632  
 -0.011063145484035312  
 -0.0710627168134462    
 -0.015356715650729887  
 -0.00024667944406771194
 -0.03474664547310121   
  0.005379190734734218  

In [15]:
cor(EBV2, y)

0.14827741122073912